# Control Ableton with OSC

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# Set the working directory to the main directory
os.chdir(os.path.dirname(os.path.abspath('')))

# Verify the current working directory
print(os.getcwd())

/home/stef/projects/ai-amg


In [3]:
from music_gen.chord_generator import ChordProgressionGenerator
from music_gen.ableton_controllers import AbletonOSCController, AbletonMetaController
# from chord_generator import ChordProgressionGenerator
from music_gen.helpers import chord_to_midi, transpose_midi_chords

In [1]:
valence = 0.5  # Example valence value between 0 and 1

# Modes dictionary
modes = {
    "lydian": [0, 2, 4, 6, 7, 9, 11],
    "ionian": [0, 2, 4, 5, 7, 9, 11],
    "mixolydian": [0, 2, 4, 5, 7, 9, 10],
    "dorian": [0, 2, 3, 5, 7, 9, 10],
    "aeolian": [0, 2, 3, 5, 7, 8, 10],
    "phrygian": [0, 1, 3, 5, 7, 8, 10],
    "locrian": [0, 1, 3, 5, 6, 8, 10],
}

# Calculate mode
mode_index = int(7 - (6 * valence)) - 1  # Subtract 1 for zero-based indexing
mode_names = list(modes.keys())         # Get mode names in order
selected_mode = mode_names[mode_index]  # Fetch the mode name
mode_notes = modes[selected_mode]       # Fetch the mode notes

# Output results
print(f"Valence: {valence}")
print(f"Selected Mode: {selected_mode}")
print(f"Mode Notes: {mode_notes}")


Valence: 0.5
Selected Mode: dorian
Mode Notes: [0, 2, 3, 5, 7, 9, 10]


In [25]:
# chord_prog = ChordProgressionGenerator()
controller = AbletonOSCController()

controller.song.set_tempo(200)

# controller.device.set_parameter(0, 0, 0, 0) # controlling frequency of filter

# prog = chord_prog.generate_progression(0.5, 0.5)
# midi_chords, midi_bass = chord_to_midi(prog)
# controller.clip.add_notes(0, 0, midi_chords)
# # controller.song.set_tempo(100) 
# arousal = -0.8 # between -1 and 1
# # Map arousal to arp rate (1-127 range)
# arp_rate = ((arousal + 1) / 2) * (127 - 1) + 1  # Rescale to 1-12
# # track number 2, first device (that is the instrument)
# controller.device.set_parameter(0, 0, 0, 1) # controlling frequency of filter

In [ ]:
from dataclasses import dataclass

@dataclass
class ChordEvent:
    notes: list     # List of MIDI notes
    duration: float # Duration in beats (assuming 4/4 time)
    velocity: float # MIDI velocity (0-1)

class MusicGenerator:
    def __init__(self):
        self.mode_intervals = [
            [0, 2, 4, 6, 7, 9, 11],  # Lydian
            [0, 2, 4, 5, 7, 9, 11],  # Ionian
            [0, 2, 4, 5, 7, 9, 10],  # Mixolydian
            [0, 2, 3, 5, 7, 9, 10],  # Dorian
            [0, 2, 3, 5, 7, 8, 10],  # Aeolian
            [0, 1, 3, 5, 7, 8, 10],  # Phrygian
            [0, 1, 3, 5, 6, 8, 10],  # Locrian
        ]
        self.fifth_order = [
            'C', 'G', 'D', 'A', 'E', 'B', 'F♯/G♭', 'C♯/D♭',
            'G♯/A♭', 'D♯/E♭', 'A♯/B♭', 'F'
        ]
        self.note_to_midi = self._build_note_to_midi_map()

    def _build_note_to_midi_map(self):
        """Create a dictionary mapping notes to MIDI numbers."""
        base_notes = {
            'C': 0, 'C♯/D♭': 1, 'D': 2, 'D♯/E♭': 3, 'E': 4, 'F': 5,
            'F♯/G♭': 6, 'G': 7, 'G♯/A♭': 8, 'A': 9, 'A♯/B♭': 10, 'B': 11
        }
        return {f"{note}{octave}": base_note + (octave * 12)
                for octave in range(0, 9)
                for note, base_note in base_notes.items()}

    def _get_chord_mode(self, valence):
        """Returns the mode intervals and notes to compose a chord."""
        mode_idx = int(7 - (6 * valence)) - 1
        mode = self.mode_intervals[mode_idx]
        indexes = [0, 2, 4, 6]
        chord_notes = [mode[i] for i in indexes]
        return mode, chord_notes

    def _construct_chord(self, tonal_midi, chord_indexes):
        chord_midi_notes = [tonal_midi + note for note in chord_indexes]
        return ChordEvent(notes=chord_midi_notes, duration=8, velocity=self.compute_velocity())            

# Example Usage
generator = MusicGenerator()
valence = 0.5
tonal_center = 'C'


# Get the chord mode
mode, chord_indexes = generator._get_chord_mode(valence)
print(mode, chord_indexes)
# # Construct the chord event
# chord_event = generator._construct_chord(tonal_center, chord_indexes)
# print(chord_event, mode, chord_indexes)


[0, 2, 3, 5, 7, 9, 10] [0, 3, 7, 10]
